In [1]:
# just the tools required to demonstrate the example 
!pip install keras 
!pip install tensorflow 

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import keras

Using TensorFlow backend.


In [3]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

In [4]:
# set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

In [5]:
import numpy as np 
np_load_old = np.load   # save old function for calling later 

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [6]:
print('Loading data...')

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [7]:
def get_fixed_word_to_id_dict(): 
    INDEX_FROM=3   # word index offset
    
    word_to_id = keras.datasets.imdb.get_word_index()
    word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
    word_to_id[" "] = 0
    word_to_id["<START>"] = 1
    word_to_id["<UNK>"] = 2
    return word_to_id

In [8]:
def decode_to_sentence(data_point): 
    NUM_WORDS=1000 # only use top 1000 words
    
    word_to_id = get_fixed_word_to_id_dict()

    id_to_word = {value:key for key,value in word_to_id.items()}
    return ' '.join(id_to_word[id] for id in data_point )

In [9]:
print(decode_to_sentence(x_train[0]))    

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly <UNK> was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little <UNK> that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big <UNK> for the whole film but these children are amazing and should be <UNK> for what they

In [10]:
print(y_train[0]) # to see the actual sentiment 

1


In [11]:
def encode_sentence(sent): 
    # print(sent)
    encoded = []
    
    word_to_id = get_fixed_word_to_id_dict() 
    
    for w in sent.split(" "): 
        if w in word_to_id: 
            encoded.append(word_to_id[w])
        else: 
            encoded.append(2)        # We used '2' for <UNK> 
    return encoded 

In [12]:
words = "fawn sonja vani made-up-word"
print(encode_sentence(words))
print(encode_sentence("this does not look good"))

[34704, 16819, 63954, 2]
[14, 127, 24, 168, 52]


In [13]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

Pad sequences (samples x time)


In [14]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters, kernel_size, padding='valid', activation='relu', strides=1))

# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)


W0811 21:46:03.098798 4652168640 deprecation_wrapper.py:119] From /Users/nam/Dropbox/Teaching/udemy/sentiment-analysis-casestudy/assets/code/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0811 21:46:03.149936 4652168640 deprecation_wrapper.py:119] From /Users/nam/Dropbox/Teaching/udemy/sentiment-analysis-casestudy/assets/code/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0811 21:46:03.163501 4652168640 deprecation_wrapper.py:119] From /Users/nam/Dropbox/Teaching/udemy/sentiment-analysis-casestudy/assets/code/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0811 21:46:03.187455 4652168640 deprecation_wrapper.py:119] From /Users/nam/Dropbox/Teaching/udemy/sentime

Build model...


W0811 21:46:03.304282 4652168640 deprecation.py:323] From /Users/nam/Dropbox/Teaching/udemy/sentiment-analysis-casestudy/assets/code/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/2
25000/25000 [==============================] - 44s 2ms/step - loss: 0.4037 - acc: 0.7999: 0s - loss: 0.4054
Epoch 2/2
25000/25000 [==============================] - 42s 2ms/step - loss: 0.2304 - acc: 0.9074


In [15]:
predictions = model.predict(x_test)
sentiment = ['NEG' if i < 0.5 else 'POS' for i in predictions]

In [16]:
data_point_to_show = 1
print(decode_to_sentence(x_test[data_point_to_show]), "--", sentiment[data_point_to_show])

                                                                                                                                                                                                                                                                                        <START> this film requires a lot of <UNK> because it focuses on mood and character development the plot is very simple and many of the scenes take place on the same set in <UNK> <UNK> the <UNK> dennis character apartment but the film builds to a disturbing climax br br the characters create an atmosphere <UNK> with sexual tension and psychological <UNK> it's very interesting that robert <UNK> directed this considering the style and structure of his other films still the <UNK> <UNK> audio style is evident here and there i think what really makes this film work is the brilliant performance by <UNK> dennis it's definitely one of her darker characters but she plays it so perfectly and convincingly that it's scary m

In [17]:
test_sentences = [] 

test_sentence = "i do not like this at all"
test_sentence = encode_sentence(test_sentence)
test_sentences.append(test_sentence) 


test_sentence = "loved it"
test_sentence = encode_sentence(test_sentence)
test_sentences.append(test_sentence) 


test_sentence = "did not love it"
test_sentence = encode_sentence(test_sentence)
test_sentences.append(test_sentence)

In [18]:
test_sentences = sequence.pad_sequences(test_sentences, maxlen=maxlen)

In [19]:
predictions = model.predict(test_sentences)
sentiment = ['NEG' if i < 0.5 else 'POS' for i in predictions]

for i in range(test_sentences.shape[0]): 
    print(decode_to_sentence(test_sentences[i]), "--", sentiment[i])

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  i do not like this at all -- NEG
                                                                                                                                                                                     